In [1]:
2

2

In [2]:
import seaborn as sns

import sklearn
from sklearn.cluster.bicluster import SpectralBiclustering
from sklearn.cluster.bicluster import SpectralCoclustering
from sklearn.metrics import consensus_score

import scipy.linalg
from functools import partial
%pylab
%matplotlib inline
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import scipy.sparse as sparse
from scipy import stats
from __future__ import print_function
import gc
from tqdm import tqdm_notebook as tqdm
from __future__ import division
import pandas as pd
import seaborn as sb
import os
from scipy.spatial import distance
from scipy.cluster import hierarchy
from pandas import DataFrame

import os, sys, datetime
LF_CODE_PATH = os.path.expanduser('~/projects/LFAnalyze/code')
FT_CODE_PATH = os.path.expanduser('~/projects/fishTrax/code/analysis/')
FD_CODE_PATH = os.path.expanduser('~/projects/fish_despair_notebooks/src/')
sys.path.insert(0,LF_CODE_PATH)
sys.path.insert(0,FT_CODE_PATH)
sys.path.insert(0,FD_CODE_PATH)

import passivity_2p_imaging_utils as p2putils
import visualization_utils as vizutil
reload(p2putils)
tmp_dir = '/tmp/'
all_data = p2putils.get_all_datasets(tmp_dir=tmp_dir)

f = all_data['e'][2]

import tensorflow as tf
# tf.enable_eager_execution()


# df, sig = f.get_cnmf_roi_table_and_signals()
M = f.get_signals_raw(z=None)

# data = np.load("../cnmf_f01555.npz")
# cnmf = data['cnmf'].astype(np.float32)
# raw = data['raw'].astype(np.float32)
# del data

def moving_average(a, n=3):
    # a is a signal
    ret = np.cumsum(a,0) # sum over time
    ret[n:] = ret[n:] - ret[:-n] # diff of n samples back
    rm = ret[n - 1:] / n
    pad_start = np.full((n-1,rm.shape[1]), rm[0])
    return np.vstack([pad_start, rm])

def ewma(data,span):
    "exponential weighted moving average."
    df = DataFrame(data)
    return df.ewm(span).mean().values

def df_f(x,ma_window=6,span=6):
    u = moving_average(x,ma_window)
    return ewma((x - u)/u, span)

# neurons = sig.T
neurons = M.T

neuron_ids = np.sort(np.argsort(neurons.std(0))[-5000:])
df = f.get_roi_table()
df = df.iloc[neuron_ids]
df.reset_index(drop=True, inplace=True)
neurons = neurons[:,neuron_ids]

/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


Using matplotlib backend: agg
Populating the interactive namespace from numpy and matplotlib


In [3]:
neurons = df_f(neurons).astype(np.float32)
neurons = (neurons - neurons.mean(0))/(neurons.std(0)+1e-8)

In [4]:
def construct_pairwise_batches(X, lag=1, startIdx=None, endIdx=None, bias=True):
    """constructs batch x nT-lag x nBeta. i*ncol+j index is granger model of influence from i->j.
    
    Use startIdx and endIdx to choose a certain number of rows,
    eg startIdx=0, endIdx=5, X.shape[1]=100 would be a batch of 100^2=10,000.
    
    WARNING: if endIdx>N, will fill with 1s so batch stays same size."""
    
    nT, N = X.shape
    if endIdx:
        bz = (endIdx - startIdx) * N
    else:
        startIdx=0
        endIdx=N
        bz = N**2
#     print([bz, nT-lag, 2*lag+bias])
    granger = np.ones([bz, nT-lag, 2*lag+bias])
    null = np.ones([bz, nT-lag, lag+bias])
    newY = np.ones([bz, nT-lag, 1])
    Y = X[lag:]
    # time x neuron x lag
    l = 0
    X_lag = np.concatenate([X[l:-(lag-l)][:,:,None] for l in reversed(range(lag))], 2)
    biasTerm = np.ones([nT-lag,1])
    for i, n1 in enumerate(range(startIdx,min(N,endIdx))):
        for j, n2 in enumerate(range(N)):
            if bias:
#                 print([x.shape for x in [X_lag[:,n1], X_lag[:,n2], biasTerm]])
                grow = np.concatenate([X_lag[:,n1], X_lag[:,n2], biasTerm], 1)
                nrow = np.concatenate([X_lag[:,n2], biasTerm], 1)
            else:
                grow = np.concatenate([X_lag[:,n1], X_lag[:,n2]], 1)
                nrow = np.concatenate([X_lag[:,n2]], 1)
#             print(i,j)
            granger[i*N+j] = grow
            null[i*N+j] = nrow
            newY[i*N+j] = Y[:,[n2]]
    return newY, granger, null

x, _, _ = construct_pairwise_batches(neurons[:,:12],1,5,15)
x.shape

(120, 2825, 1)

In [6]:
def f_value(resA, resB, n_params_A, n_params_B, n):
    return ( (resA-resB)/(n_params_B-n_params_A) ) / (resB/(n-n_params_B))

class Granger():
    def __init__(self, x_granger_shape, x_null_shape, y_shape, sess, l2=0.0):
        self.x_granger = tf.placeholder(dtype=tf.float32, shape=x_granger_shape)
        self.x_null = tf.placeholder(dtype=tf.float32, shape=x_null_shape)
        self.y = tf.placeholder(dtype=tf.float32, shape=y_shape)
        self.n_params_granger = x_granger_shape[-1]
        self.n_params_null = x_null_shape[-1]
        self.l2 = l2
        self.sess = sess
        
    def granger(self, X, Y):
        gbeta = tf.matrix_solve_ls(self.x_granger, self.y, l2_regularizer=self.l2, fast=False)
        g_residuals = tf.reduce_sum(((tf.matmul(self.x_granger,gbeta) - self.y)**2)[:,:,0], 1)
        gbeta, g_residuals = self.sess.run([gbeta, g_residuals],
                                      feed_dict={self.x_granger: X, self.y: Y})
        return gbeta, g_residuals
    
    def null(self, X, Y):
        gbeta = tf.matrix_solve_ls(self.x_null, self.y, l2_regularizer=self.l2, fast=False)
        g_residuals = tf.reduce_sum(((tf.matmul(self.x_null,gbeta) - self.y)**2)[:,:,0], 1)
        gbeta, g_residuals = self.sess.run([gbeta, g_residuals],
                                      feed_dict={self.x_null: X, self.y: Y})
        return gbeta, g_residuals

def pairwise_granger_f_val(neurons, lag=1, bz=10):
    nT, N = neurons.shape
    Y, x_granger, x_null = construct_pairwise_batches(neurons,lag,0,bz)
    fvals = np.zeros([N,N])
    with tf.Session() as sess:
        granger = Granger(x_granger.shape, x_null.shape, Y.shape, sess=sess, l2=0.0)
        for start in tqdm(range(0,N,bz)):
            end = start+bz
            Y, x_granger, x_null = construct_pairwise_batches(neurons,1,start,end)
            gbeta, g_residuals = granger.granger(x_granger, Y)
            nbeta, n_residuals = granger.null(x_null, Y)
            batch_fvals = f_value(g_residuals, n_residuals, granger.n_params_granger, granger.n_params_null, N)
            true_end = min(start+bz,N)
            batch_fvals = batch_fvals.reshape(-1,N)
            if start+bz>N:
                batch_true_end = N-start
                batch_fvals = batch_fvals[:batch_true_end]
            fvals[start:true_end] = batch_fvals
    return fvals

f_vals = pairwise_granger_f_val(neurons, 1, 10)

plt.imshow(f_vals)
plt.colorbar()

InvalidArgumentError: Cannot assign a device for operation 'MatrixSolveLs': Could not satisfy explicit device specification '/device:GPU:0' because no supported kernel for GPU devices is available.
Registered kernels:
  device='CPU'; T in [DT_FLOAT]
  device='CPU'; T in [DT_DOUBLE]
  device='CPU'; T in [DT_COMPLEX64]
  device='CPU'; T in [DT_COMPLEX128]

	 [[{{node MatrixSolveLs}} = MatrixSolveLs[T=DT_FLOAT, fast=true, _device="/device:GPU:0"](Placeholder, Placeholder_2, MatrixSolveLs/l2_regularizer)]]

Caused by op u'MatrixSolveLs', defined at:
  File "/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ubun

In [17]:
batch_size = 10
Y, x_granger, x_null = construct_pairwise_batches(neurons,1,0,batch_size)

In [98]:
# tf init
granger = Granger(x_granger.shape, x_null.shape, Y.shape, l2=0.0)

In [99]:
gbeta, g_residuals = granger.granger(x_granger, Y)
nbeta, n_residuals = granger.null(x_null, Y)

np_beta, _ = np.linalg.lstsq(x_granger[0], Y[0], rcond=None)[0:2]
np_res = sum((np.matmul(x_granger[0],np_beta) - Y[0])**2)
print("tf vs numpy")
print(gbeta[0,:,0],"\n", np_beta[:,0])
print("residuals", float(g_residuals[0]), np_res)

tf vs numpy
[ 8.3156904e+03 -8.3149883e+03  1.9857315e-03] 
 [0.34207702 0.34207702 0.00227213]
residuals 1461.8347168 1460.9055601827695


InvalidArgumentError: Cannot assign a device for operation 'MatrixSolveLs_114': Could not satisfy explicit device specification '/device:GPU:1' because no supported kernel for GPU devices is available.
Registered kernels:
  device='CPU'; T in [DT_FLOAT]
  device='CPU'; T in [DT_DOUBLE]
  device='CPU'; T in [DT_COMPLEX64]
  device='CPU'; T in [DT_COMPLEX128]

	 [[Node: MatrixSolveLs_114 = MatrixSolveLs[T=DT_FLOAT, fast=false, _device="/device:GPU:1"](Placeholder_140, Placeholder_142, MatrixSolveLs_114/l2_regularizer)]]

Caused by op u'MatrixSolveLs_114', defined at:
  File "/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance(

In [121]:
import gc

gc.collect()

T.cuda.empty_cache()

In [ ]:
def cm_cycle(i):
    i = i - 1
    cmap = plt.get_cmap("tab10")
    color = cmap(i%10)
    if i%10==7:
        color = (color[0],color[1],color[2]+0.4)
    return color

In [ ]:
row_corr = corrcoef(G).cpu().numpy()

row_linkage = hierarchy.linkage(
    distance.pdist(row_corr), method='ward')

In [ ]:
col_corr = corrcoef(G.t().detach()).cpu().numpy()
col_linkage = hierarchy.linkage(
    distance.pdist(col_corr.T), method='ward')

In [ ]:
g_nclust = 10
row_clusters = hierarchy.fcluster(row_linkage,g_nclust,criterion='maxclust')
col_clusters = hierarchy.fcluster(col_linkage,g_nclust,criterion='maxclust')

In [ ]:
vmax = max(np.abs(np.percentile(G,0.01)), np.abs(np.percentile(G,0.99)))

In [ ]:
cm = sb.clustermap(G.cpu().numpy(), row_linkage=row_linkage, row_colors=[cm_cycle(c) for c in row_clusters],
        col_linkage=col_linkage, col_colors=[cm_cycle(c) for c in col_clusters],
        figsize=(20, 20),cmap="RdBu_r", vmin=-vmax,vmax=vmax)

In [ ]:
Z = [0,2,4,6,8,10]
# Z = [4]
nZ = len(Z)
back_img = []
for z in Z:
    back_img.append(np.power(f.get_tif_rasl_as_vol(z,range(1,200)).mean(axis=2),.4))
    

In [ ]:
# nrows = len(row_clust_to_plot)
# nZ = 1
nrows = g_nclust
plt.subplots(nrows,nZ, figsize=[8*nZ,4*nrows])
for ic, clust in enumerate(range(1, g_nclust+1)):
# for ic,clust in enumerate(row_clust_to_plot):
    for iz, z in enumerate(Z):
        #Select rois in raphe in this slices, and get their coordinates.
#         coords = df[(row_clusters==clust)].coords
        coords = df[(row_clusters==clust) & (df.z==z)].coords
#         poly_coords = df[(row_clusters==clust) & (df.z==z)].poly
#         poly_coords = df[(row_clusters==clust)].poly
#         coords = [np.round(poly[~np.isnan(poly).any(axis=1)])[:,(1,0)].astype(int) for poly in poly_coords]
        plt.subplot(nrows,nZ,ic*nZ+iz+1)
        #Overlay the ROIs on the background image and display:
        # hack iz hardcode
        img = vizutil.overlay_coords(back_img[iz], coords, list(cm_cycle(clust)[:3]), alpha=1)
        plt.imshow(img,interpolation='nearest')
        plt.title("Row cluster {}, z={}".format(clust,z),fontsize=18)

In [ ]:
# nrows = len(col_clust_to_plot)
plt.subplots(nrows,nZ, figsize=[8*nZ,4*nrows])
for ic,clust in enumerate(range(1,g_nclust+1)):
# for ic,clust in enumerate(col_clust_to_plot):
    for iz, z in enumerate(Z):
        #Select rois in raphe in this slices, and get their coordinates.
        coords = df[(col_clusters==clust) & (df.z==z)].coords
#         poly_coords = df[(row_clusters==clust) & (df.z==z)].poly
#         coords = df[(col_clusters==clust)].coords
#         poly_coords = df[(col_clusters==clust)].poly
#         coords = [np.round(poly[~np.isnan(poly).any(axis=1)])[:,(1,0)].astype(int) for poly in poly_coords]
        plt.subplot(nrows,nZ,ic*nZ+iz+1)
        #Overlay the ROIs on the background image and display:
        img = vizutil.overlay_coords(back_img[iz], coords, list(cm_cycle(clust)[:3]), alpha=1)
        plt.imshow(img,interpolation='nearest')
        plt.title("Col cluster {}, z={}".format(clust,z),fontsize=18)

In [ ]:
s = 0
e = 440
fig, ax = plt.subplots(1,2, figsize=(5,100))
ax[0].imshow(X[slice(*idx_permute(s,e,exclude_idx))].T)
ax[1].imshow(x_fish[s:e].T)

In [ ]:
colidx = col_clusters==34
rowidx = row_clusters==7
plt.matshow(G[rowidx][:,colidx],vmin=-0.1,vmax=0.1)
plt.colorbar()

In [ ]:
ind = slice(*idx_permute(2400,2800, exclude_idx))
# ind = slice(*idx_permute(0,400, exclude_idx))

In [ ]:
# model = Granger(len(neuron_ids),1,bias=False)
model = Granger(X.shape[1],1,bias=False)
G = model.G.data[0].detach().cpu().numpy()
# trunc_fish = x_fish[0:400]
# trunc_fish = x_fish[440:840]
# trunc_fish = x_fish[840:1240]
# trunc_fish = x_fish[1240:1640]
# trunc_fish = x_fish[1640:2040]
# trunc_fish = x_fish[1640:2440]
# trunc_fish = x_fish[640:840]
# trunc_fish = x_fish[2400:2800]
# trunc_fish = x_fish[2600:2800]
# trunc_fish = x_fish
# data = T.utils.data.TensorDataset(T.from_numpy(X[ind,None,neuron_ids]).cuda(),
#                                   T.from_numpy(Y[ind,neuron_ids]).cuda())
data = T.utils.data.TensorDataset(T.from_numpy(X[ind,None]).cuda(),
                                  T.from_numpy(Y[ind]).cuda())
l1 = 5e-6
# train(model,data,10,l1=l1,batch_size=4) # max(bz)==4 for all neurons
train(model,data,3,l1p=1e-0,lr=1e-1,batch_size=1,verbose=False)
train(model,data,15,l1p=5e-2,lr=1e-3,batch_size=1,verbose=False)
G = model.G.data[0].detach().cpu().numpy()